In [1]:
import requests
from bs4 import BeautifulSoup

url = requests.get("https://id.arduino.cc/auth/login/")
soup = BeautifulSoup(url.content, "html.parser")
# soup

In [2]:
data = soup.find("h2",{"class":"display-post-title"})
# data

In [7]:

import signal


class TimeoutException(Exception):
    pass


class Timeout(object):
    """Timeout class using ALARM signal
    This can be used as a context manager to raise an ALARM signal at a
    specified time in the future, that alarm is then used to raise a
    TimeoutException
    """
    def __init__(self, sec):
        self.sec = int(sec)

    def __enter__(self):
        signal.signal(signal.SIGALRM, self.raise_timeout)
        signal.alarm(self.sec)

    def __exit__(self, *args):
        signal.alarm(0)  # disable alarm

    def raise_timeout(self, *args):
        raise TimeoutException()

In [18]:
# from httplib import BadStatusLine
import logging
import mechanize
import socket
import time
# from urllib3 import HTTPError, URLError
import urllib3

# from timeout import Timeout, TimeoutException

try:  # Python 2.7+
    from logging import NullHandler
except ImportError:
    class NullHandler(logging.Handler):
        def emit(self, record):
            pass
    logging.NullHandler = NullHandler


logger = logging.getLogger("mechanizeretry")
logger.addHandler(logging.NullHandler())


class RetryBrowser(mechanize.Browser):
    """ A sublcass of the mechanize.Browser class that adds retries if an
        HTTPError, URLError or BadStatusLine occurs to the following methods:
        * open
        * submit
        If the methods are called without retries specified, they default to
        the standard mechanize.Browser methods.  If retries are specified,
        the method standard calls are wrapped in a loop
    """
    def __init__(self, global_timeout=None, **kwargs):
        """
        :param global_timeout: int, defaults to 300 if not set.
        :param kwargs:
        """
        self._global_timeout = global_timeout if global_timeout is not None else 300
        mechanize._sockettimeout._GLOBAL_DEFAULT_TIMEOUT = self.global_timeout
        mechanize.Browser.__init__(self, **kwargs)

    @property
    def global_timeout(self):
        return self._global_timeout

    def _open(self, url, data=None, headers=None,
              timeout=None):
        timeout = self.global_timeout if timeout is None else timeout
        # mechanize calls open internally with request objects, just pass them through if this is
        # the case.
        if isinstance(url, mechanize.Request):
            request = url
        else:
            headers = dict() if headers is None else headers
            request = mechanize.Request(url, data=data, headers=headers, timeout=timeout)
        with Timeout(timeout):
            return mechanize.Browser.open(self, request)

    def open(self, url, data=None, headers=None, retries=None, delay=None, backoff=None,
             timeout=None):
        timeout = self.global_timeout if timeout is None else timeout
        # Reset to sane values if needed.
        retries = 1 if retries is None or retries == 0 else retries
        delay = 0 if delay is None else delay
        print(delay)
        backoff = 1 if backoff is None or backoff == 0 else backoff
        # Retry with sleep on certain errors
        for atry in range(1, retries + 1):
            try:
                return self._open(url, data=data, headers=headers, timeout=timeout)
            except (urllib3.exceptions.HTTPError, socket.timeout, socket.error) as e:
                logger.error("Open of '{0}' failed - Error: {1}".format(url, e))
                try:
                    logger.error("Response:\n{0}".format(e.read()))
                except AttributeError:
                    pass
                if atry == retries:
                    raise
            except urllib3.exceptions.URLError as e:
                logger.error("Open of '{0}' failed - Error: {1}".format(url, e))
                if 'Connection timed out' not in e.reason:
                    # Raise URLError right away if the error is not a timeout
                    logger.error("Open of '{0}' failed - Error: {1}".format(url, e))
                    raise
                if atry == retries:
                    raise
            except TimeoutException:
                logger.error("Open request to {0} timed out".format(url))
                if atry == retries:
                    raise TimeoutException("Open request to {0} timed out".format(url))
            delay = delay * backoff
            if backoff != 1:
                logger.info("Backed delay off by a factor of {0} to {1}".format(backoff, delay))
            time.sleep(delay)

    def _submit(self, timeout=None, *args, **kwargs):
        timeout = self.global_timeout if timeout is None else timeout
        with Timeout(timeout):
            return mechanize.Browser.submit(self, *args, **kwargs)

    def submit(self, retries=None, delay=None, backoff=None, timeout=None, *args, **kwargs):
        timeout = self.global_timeout if timeout is None else timeout
        # Reset for sane values
        retries = 1 if retries is None or retries == 0 else retries
        delay = 0 if delay is None else delay
        backoff = 1 if backoff is None or backoff == 0 else backoff
        # Retry with sleep on HTTPError, URLError and BadStatusLine
        for atry in range(1, retries + 1):
            try:
                return self._submit(timeout=timeout, *args, **kwargs)
            except (urllib3.exceptions.HTTPError, urllib3.exceptions.URLError) as e:
                logger.error("Submit failed - Error: {0}".format(e))
                try:
                    logger.error("Response:\n{0}".format(e.read()))
                except AttributeError:
                    pass
                if atry == retries:
                    raise
            except TimeoutException:
                logger.error("Submit request timed out")
                if atry == retries:
                    raise TimeoutException("Submit request timed out")
            delay = delay * backoff
            if backoff != 1:
                logger.info("Backed delay off by a factor of {0} to {1}".format(backoff, delay))
            time.sleep(delay)

    def _select_form(self, timeout=None, *args, **kwargs):
        timeout = self.global_timeout if timeout is None else timeout
        with Timeout(timeout):
            return mechanize.Browser.select_form(self, *args, **kwargs)

    def select_form(self, retries=None, delay=None, backoff=None, timeout=None, *args, **kwargs):
        timeout = self.global_timeout if timeout is None else timeout
        # Reset for sane values
        retries = 1 if retries is None or retries == 0 else retries
        delay = 0 if delay is None else delay
        backoff = 1 if backoff is None or backoff == 0 else backoff
        for atry in range(1, retries + 1):
            try:
                return self._select_form(timeout, *args, **kwargs)
            except TimeoutException:
                logger.error("select_form request timed out")
                if atry == retries:
                    raise TimeoutException("select_form request timed out")
            delay = delay * backoff
            if backoff != 1:
                logger.info("Backed delay off by a factor of {0} to {1}".format(backoff, delay))
            time.sleep(delay)

In [29]:
import mechanize
import requests
from bs4 import BeautifulSoup
import urllib3
# import cookielib ## http.cookiejar in python3
import http.cookiejar


cookielib = http.cookiejar
cj = cookielib.CookieJar()
br = RetryBrowser()
br.set_cookiejar(cj)
br.open("https://happywork.jenosize.com/admin/my-dashboard/")

br.select_form(nr=0)
br.form['email'] = 'suthida.b@codium.co'
br.form['password'] = 'Suthida2540'
br.submit()

br.open("https://happywork.jenosize.com/admin/attendance-dashboard")
# br.find_link("https://happywork.jenosize.com/admin/attendance-dashboard")


0
0
0
0
0
0
300000


<response_seek_wrapper at 0x7f99a6e23550 whose wrapped object = <closeable_response at 0x7f99a6db3e50 whose fp = <_io.BufferedReader name=73>>>

In [27]:
soup = BeautifulSoup(br.response().read())
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>HappyWork | หน้ากระดานการเข้า - ออกงาน</title>
<link href="https://happywork.jenosize.com/cms_admin/css/bootstrap.min.css" rel="stylesheet"/>
<link href="https://happywork.jenosize.com/cms_admin/font-awesome/css/font-awesome.css" rel="stylesheet"/>
<link href="https://happywork.jenosize.com/cms_admin/css/plugins/iCheck/custom.css" rel="stylesheet"/>
<link href="https://happywork.jenosize.com/cms_admin/css/plugins/chosen/bootstrap-chosen.css" rel="stylesheet"/>
<link href="https://happywork.jenosize.com/cms_admin/css/plugins/bootstrap-tagsinput/bootstrap-tagsinput.css" rel="stylesheet"/>
<link href="https://happywork.jenosize.com/cms_admin/css/plugins/colorpicker/bootstrap-colorpicker.min.css" rel="stylesheet"/>
<link href="https://happywork.jenosize.com/cms_admin/css/plugins/cropper/cropper.min.css" r

In [28]:
soup.find('table', id="tbl_attendance")

<table class="table table-striped table-bordered table-hover dataTables-attendance" id="tbl_attendance" style="width:100%;">
<thead>
<tr>
<th class="text-center" style="min-width: 10%; vertical-align: middle;">รหัสพนักงาน</th>
<th class="text-center" style="min-width: 20%; vertical-align: middle;">ชื่อพนักงาน</th>
<th class="text-center" style="width: 30%; vertical-align: middle;">เข้างาน</th>
<th class="text-center" style="width: 30%; vertical-align: middle;">ออกงาน</th>
<th class="text-center" style="width: 10%; vertical-align: middle;">บันทึกการทำงาน</th>
</tr>
</thead>
</table>

In [ ]:
"""Username: suthida.b@codium.co
Password: Suthida2540"""

Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""optimizelyEndUserId=oeu1614568690578r0.429045785466289; ajs_anonymous_id=%224c9d15d3-272d-41d0-a19e-f79521509b15%22; _gid=GA1.2.805099609.1614568694; _hjTLDTest=1; _hjid=59068fe3-e921-403b-a8f0-03978ac6d86f; _hjIncludedInPageviewSample=1; _hjAbsoluteSessionInProgress=1; _gat_UA-122917547-2=1; _ga=GA1.2.1921975258.1614568694; XSRF-TOKEN=eyJpdiI6Indna0xcL05qeU9RM3V5TXR6RGNaTDZRPT0iLCJ2YWx1ZSI6ImZ2bGFmSmRlQXRRN25vOEVDdzlCYjBRSFJGMGYyZHVRTVdpYjJrdmdiXC8xMndMQkpUcVJFS3k5WHRJR04zdE5HRE5EQzNOMVIxZjZkditEU0NvM0l0QT09IiwibWFjIjoiZTFkMTBkNzAxNGQ0NDFjZjYxMTU1ZDRhZmYwNTk1ZTliOGRlNGY4NzVkODE0ZDVlMjdhNWM0ODMxYjViOTYwOSJ9; laravel_session=eyJpdiI6IkU4MHNTMFJTQ01PVTNLS1g0TWVJbkE9PSIsInZhbHVlIjoiVjh6VUQrSDRDUzlrZGhEVUd2U3c4MzJoNFpoVXlvTTNpUW1IdGd6OWZvK3hQbm1sNFhGdkdvUzZ2bGtncUdDdTkxVnduSXlFYVJpTUxKY0VcL3ZjQUxBPT0iLCJtYWMiOiJlMWNkYjI1MjFhZGNmYTE3MGJlMmNlMWM4OWYwN2QxNTAxNDQwOGZlZGU3ZjFmYjI3ZDcwYzEwMjk1Njg2ZWQ1In0%3D; _ga_H9V8750GZD=GS1.1.1614672159.5.1.1614673260.0"
""

ไอเดีย อยากรู้ว่าในอนาคต เทรนของ lead จะเปนยังไง
ใน 1 lead ใช้เวลาปิดเท่าไหร่ ในอนาคต

In [30]:
pip install tensorflow

     |████████████████████████████████| 394.4 MB 9.9 kB/s  eta 0:00:01  |▏                               | 2.3 MB 1.9 MB/s eta 0:03:26     |▍                               | 4.4 MB 1.9 MB/s eta 0:03:25     |▋                               | 7.4 MB 1.9 MB/s eta 0:03:23     |▉                               | 10.4 MB 5.6 MB/s eta 0:01:09     |█▏                              | 14.6 MB 5.6 MB/s eta 0:01:08     |█▎                              | 15.6 MB 5.6 MB/s eta 0:01:08     |██▏                             | 26.4 MB 6.3 MB/s eta 0:00:59     |██▍                             | 29.8 MB 14.1 MB/s eta 0:00:26     |███▉                            | 47.7 MB 7.7 MB/s eta 0:00:46     |████                            | 48.5 MB 7.7 MB/s eta 0:00:46     |████                            | 50.4 MB 3.8 MB/s eta 0:01:31     |████▏                           | 50.9 MB 3.8 MB/s eta 0:01:31     |████▏                           | 51.1 MB 3.8 MB/s eta 0:01:31     |████▎                           | 52.2 MB 3.8

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=8dfe7bc60dae2ac321eb09a2bfaddd32e84e1f5a00de8f5ec2b417019600072e
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl size=81509 sha256=f5b5442dccafb594dea046f826e1c89b8bc997988ac3707c30367db6953e6457
  Stored in directory: /root/.cache/pip/wheels/5f/fd/9e/b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built termcolor wrapt
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.35.0
    Uninstalling grpcio-1.35.0:
      Successfully uninstalled grpcio-1.35.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install tensorflow-gpu

     |██████████████████████████████▎ | 373.8 MB 11.6 MB/s eta 0:00:02  |                                | 634 kB 2.2 MB/s eta 0:03:03     |▏                               | 1.6 MB 2.2 MB/s eta 0:03:03     |▋                               | 7.1 MB 2.2 MB/s eta 0:03:00     |█▋                              | 19.5 MB 9.7 MB/s eta 0:00:39     |██                              | 23.3 MB 9.7 MB/s eta 0:00:39     |██                              | 24.3 MB 9.7 MB/s eta 0:00:39     |██▎                             | 27.7 MB 13.1 MB/s eta 0:00:29     |███▉                            | 47.1 MB 19.2 MB/s eta 0:00:19     |█████▏                          | 63.4 MB 10.0 MB/s eta 0:00:34     |█████▍                          | 66.3 MB 10.0 MB/s eta 0:00:33     |█████▌                          | 67.2 MB 10.0 MB/s eta 0:00:33     |█████▉                          | 72.6 MB 10.0 MB/s eta 0:00:33     |████████████                    | 147.9 MB 13.3 MB/s eta 0:00:19     |████████████▌                   | 153.